# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 2 2022 10:13AM,249867,10,0086138933,ISDIN Corporation,Released
1,Nov 2 2022 10:13AM,249867,10,0086138941,ISDIN Corporation,Released
2,Nov 2 2022 10:13AM,249867,10,0086138938,ISDIN Corporation,Released
3,Nov 2 2022 10:04AM,249866,10,0086138927,ISDIN Corporation,Released
4,Nov 2 2022 10:04AM,249866,10,0086138928,ISDIN Corporation,Released
5,Nov 2 2022 9:53AM,249865,15,PNC391437A,"Person & Covey, Inc.",Executing
6,Nov 2 2022 9:53AM,249865,15,PNC398094A,"Person & Covey, Inc.",Released
7,Nov 2 2022 9:53AM,249865,15,PNC398114A,"Person & Covey, Inc.",Released
8,Nov 2 2022 9:53AM,249865,15,PNC394505B,"Person & Covey, Inc.",Released
9,Nov 2 2022 9:53AM,249865,15,PNC397995B,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,249864,Released,1
25,249865,Executing,2
26,249865,Released,5
27,249866,Released,2
28,249867,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249862,NaN,NaN,15.0
249864,NaN,NaN,1.0
249865,NaN,2.0,5.0
249866,NaN,NaN,2.0
249867,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249794,0.0,0.0,1.0
249795,0.0,0.0,1.0
249807,0.0,11.0,4.0
249828,0.0,0.0,1.0
249831,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249794,0,0,1
249795,0,0,1
249807,0,11,4
249828,0,0,1
249831,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249794,0,0,1
1,249795,0,0,1
2,249807,0,11,4
3,249828,0,0,1
4,249831,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249794,,,1
1,249795,,,1
2,249807,,11,4
3,249828,,,1
4,249831,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 2 2022 10:13AM,249867,10,ISDIN Corporation
3,Nov 2 2022 10:04AM,249866,10,ISDIN Corporation
5,Nov 2 2022 9:53AM,249865,15,"Person & Covey, Inc."
12,Nov 2 2022 9:48AM,249864,15,"Alliance Pharma, Inc."
13,Nov 2 2022 9:43AM,249856,15,"Alliance Pharma, Inc."
43,Nov 2 2022 9:38AM,249862,15,"Virtus Pharmaceuticals, LLC"
58,Nov 2 2022 9:35AM,249861,15,"Mizner Bioscience, LLC"
66,Nov 2 2022 9:27AM,249858,10,Eywa Pharma Inc.
78,Nov 2 2022 9:26AM,249860,10,ISDIN Corporation
97,Nov 2 2022 9:26AM,249857,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 2 2022 10:13AM,249867,10,ISDIN Corporation,,,3
1,Nov 2 2022 10:04AM,249866,10,ISDIN Corporation,,,2
2,Nov 2 2022 9:53AM,249865,15,"Person & Covey, Inc.",,2,5
3,Nov 2 2022 9:48AM,249864,15,"Alliance Pharma, Inc.",,,1
4,Nov 2 2022 9:43AM,249856,15,"Alliance Pharma, Inc.",,,30
5,Nov 2 2022 9:38AM,249862,15,"Virtus Pharmaceuticals, LLC",,,15
6,Nov 2 2022 9:35AM,249861,15,"Mizner Bioscience, LLC",,,8
7,Nov 2 2022 9:27AM,249858,10,Eywa Pharma Inc.,,,12
8,Nov 2 2022 9:26AM,249860,10,ISDIN Corporation,,,19
9,Nov 2 2022 9:26AM,249857,15,"Brookfield Pharmaceuticals, LLC",,,28


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 10:13AM,249867,10,ISDIN Corporation,3,,
1,Nov 2 2022 10:04AM,249866,10,ISDIN Corporation,2,,
2,Nov 2 2022 9:53AM,249865,15,"Person & Covey, Inc.",5,2,
3,Nov 2 2022 9:48AM,249864,15,"Alliance Pharma, Inc.",1,,
4,Nov 2 2022 9:43AM,249856,15,"Alliance Pharma, Inc.",30,,
5,Nov 2 2022 9:38AM,249862,15,"Virtus Pharmaceuticals, LLC",15,,
6,Nov 2 2022 9:35AM,249861,15,"Mizner Bioscience, LLC",8,,
7,Nov 2 2022 9:27AM,249858,10,Eywa Pharma Inc.,12,,
8,Nov 2 2022 9:26AM,249860,10,ISDIN Corporation,19,,
9,Nov 2 2022 9:26AM,249857,15,"Brookfield Pharmaceuticals, LLC",28,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 10:13AM,249867,10,ISDIN Corporation,3,,
1,Nov 2 2022 10:04AM,249866,10,ISDIN Corporation,2,,
2,Nov 2 2022 9:53AM,249865,15,"Person & Covey, Inc.",5,2,
3,Nov 2 2022 9:48AM,249864,15,"Alliance Pharma, Inc.",1,,
4,Nov 2 2022 9:43AM,249856,15,"Alliance Pharma, Inc.",30,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 10:13AM,249867,10,ISDIN Corporation,3.0,NaN,NaN
1,Nov 2 2022 10:04AM,249866,10,ISDIN Corporation,2.0,NaN,NaN
2,Nov 2 2022 9:53AM,249865,15,"Person & Covey, Inc.",5.0,2.0,NaN
3,Nov 2 2022 9:48AM,249864,15,"Alliance Pharma, Inc.",1.0,NaN,NaN
4,Nov 2 2022 9:43AM,249856,15,"Alliance Pharma, Inc.",30.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 10:13AM,249867,10,ISDIN Corporation,3.0,0.0,0.0
1,Nov 2 2022 10:04AM,249866,10,ISDIN Corporation,2.0,0.0,0.0
2,Nov 2 2022 9:53AM,249865,15,"Person & Covey, Inc.",5.0,2.0,0.0
3,Nov 2 2022 9:48AM,249864,15,"Alliance Pharma, Inc.",1.0,0.0,0.0
4,Nov 2 2022 9:43AM,249856,15,"Alliance Pharma, Inc.",30.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1998827,71.0,6.0,1.0
12,249849,22.0,0.0,0.0
15,1749002,89.0,2.0,0.0
19,1249178,12.0,13.0,0.0
21,499589,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1998827,71.0,6.0,1.0
1,12,249849,22.0,0.0,0.0
2,15,1749002,89.0,2.0,0.0
3,19,1249178,12.0,13.0,0.0
4,21,499589,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,71.0,6.0,1.0
1,12,22.0,0.0,0.0
2,15,89.0,2.0,0.0
3,19,12.0,13.0,0.0
4,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,71.0
1,12,Released,22.0
2,15,Released,89.0
3,19,Released,12.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21
Status,,,,,
Completed,1.0,0.0,0.0,0.0,0.0
Executing,6.0,0.0,2.0,13.0,0.0
Released,71.0,22.0,89.0,12.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,2.0,13.0,0.0
2,Released,71.0,22.0,89.0,12.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,2.0,13.0,0.0
2,Released,71.0,22.0,89.0,12.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()